In [109]:
import json
import pandas as pd
from sklearn.metrics import pairwise_distances

In [110]:
spotify_artists = pd.read_csv('/Users/nooshy/Downloads/archive (7)/nodes.csv')

spotify_artists.head()

,spotify_id,name,followers,popularity,genres,chart_hits
0,48WvrUGoijadXXCsGocwM4,Byklubben,1738.0,24.0,"['nordic house', 'russelater']",['no (3)']
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72.0,"['christlicher rap', 'german hip hop']","['at (44)', 'de (111)', 'lu (22)', 'ch (31)', ..."
2,652XIvIBNGg3C0KIGEJWit,Maxim,34596.0,36.0,[],['de (1)']
3,3dXC1YPbnQPsfHPVkm1ipj,Christopher Martin,249233.0,52.0,"['dancehall', 'lovers rock', 'modern reggae', ...","['at (1)', 'de (1)']"
4,74terC9ol9zMo8rfzhSOiG,Jakob Hellman,21193.0,39.0,"['classic swedish pop', 'norrbotten indie', 's...",['se (6)']


In [111]:
#Cleaning:
#Remove whitespace
spotify_artists.columns = spotify_artists.columns.str.strip()

#Drop any nas
spotify_artists = spotify_artists.dropna(subset = ['spotify_id', 'name', 'genres'])

In [173]:
import ast 
artist_genre = {}

# Iterate over each row in the DataFrame
for index, row in spotify_artists.iterrows():
    artist_id = row['spotify_id']
    name = row['name']
    popularity = row['popularity']
    genres = row['genres']

    # Check if 'genres' is a string and convert to list if needed
    if isinstance(genres, str):
        genres = ast.literal_eval(genres) 

    if artist_id not in artist_genre:
        artist_genre[artist_id] = {}

    for genre in genres:
        genre = genre.strip()
        if genre not in artist_genre[artist_id]:
            artist_genre[artist_id][genre] = 0 
        artist_genre[artist_id][genre] += 1

# Convert the dictionary to a DataFrame
artist_genre_df = pd.DataFrame.from_dict(artist_genre, orient='index').fillna(0)
print(spotify_artists['genres'].head())

0                       ['nordic house', 'russelater']
1               ['christlicher rap', 'german hip hop']
2                                                   []
3    ['dancehall', 'lovers rock', 'modern reggae', ...
4    ['classic swedish pop', 'norrbotten indie', 's...
Name: genres, dtype: object


In [195]:
query_artist_1 = '48WvrUGoijadXXCsGocwM4'

# Select the query artist row from the DataFrame
query_artist1 = artist_genre_df.loc[[query_artist_1]] 

# Align both the query_artist and artist_genre_df to have the same columns
query1_artist_aligned = query_artist1.reindex(columns=artist_genre_df.columns, fill_value=0)
artist_genre_df_aligned = artist_genre_df.reindex(columns=artist_genre_df.columns, fill_value=0)

# Euclidean distance calculations
query1_euclidean_distances = pairwise_distances(query1_artist_aligned, artist_genre_df_aligned, metric='euclidean')[0]
query1_euclidean_top10_indices = query1_euclidean_distances.argsort()[:11]

id_to_name = spotify_artists.set_index('spotify_id')['name'].to_dict()

print("Top 10 Artists Most Similar to Byklubben, based on genre: \n")

for idx in query1_euclidean_top10_indices:
    artist_id = artist_genre_df.index[idx] 
    artist_name = id_to_name.get(artist_id, "Unknown Artist")  
    distance = query1_euclidean_distances[idx]  
    print(f"Artist: {artist_name} (ID: {artist_id}), Distance: {distance}")

Top 10 Artists Most Similar to Byklubben, based on genre: 

Artist: Byklubben (ID: 48WvrUGoijadXXCsGocwM4), Distance: 0.0
Artist: Klikkmonopolet (ID: 49801AhfB2xOoOqtRZXv4H), Distance: 1.0
Artist: J-Dawg (ID: 18KsILOMx3thJRzBYBnKHW), Distance: 1.0
Artist: b.G (ID: 2zw4zwwb2LoNYssj3hdLoa), Distance: 1.0
Artist: Snuskebazn (ID: 62InFTSRpk70tkIgzjLNTR), Distance: 1.0
Artist: Heux (ID: 7IuE8jJQzeifyrcFYdrIWt), Distance: 1.0
Artist: Brødrene Bausa (ID: 7krakxeyEmge6pzTthKNyg), Distance: 1.0
Artist: DJ Smaaland (ID: 5ksD5XT78Q3JpED82u9rNO), Distance: 1.0
Artist: Mad.S  (ID: 1rQmmtMZFDzOadozdmep2k), Distance: 1.0
Artist: Hanzee (ID: 5yM1po4NHvE2yE1Kf84LWJ), Distance: 1.0
Artist: Moberg (ID: 0l8OJzLFIV1OqbU4OhcZLu), Distance: 1.0


In [196]:
query_artist_2 = '3dXC1YPbnQPsfHPVkm1ipj'

# Select the query artist row from the DataFrame
query_artist2 = artist_genre_df.loc[[query_artist_2]]

# Align both the query_artist and artist_genre_df to have the same columns
query2_artist_aligned = query_artist2.reindex(columns=artist_genre_df.columns, fill_value=0)

# Euclidean distance calculations
query2_euclidean_distances = pairwise_distances(query2_artist_aligned, artist_genre_df_aligned, metric='euclidean')[0]
query2_euclidean_top10_indices = query2_euclidean_distances.argsort()[:11]

print("Top 10 Artists Most Similar to Christopher Martin, based on genre: \n")

for idx in query2_euclidean_top10_indices:
    artist_id = artist_genre_df.index[idx]  
    artist_name = id_to_name.get(artist_id, "Unknown Artist") 
    distance = query2_euclidean_distances[idx]
    print(f"Artist: {artist_name} (ID: {artist_id}), Distance: {distance}")

Top 10 Artists Most Similar to Christopher Martin, based on genre: 

Artist: Christopher Martin (ID: 3dXC1YPbnQPsfHPVkm1ipj), Distance: 0.0
Artist: Protoje (ID: 7BGR8y1VZAWK2oR4zD9COr), Distance: 1.4142135623730951
Artist: MAGIC! (ID: 0DxeaLnv6SyYk2DOqkLO8c), Distance: 1.7320508075688772
Artist: Vybz Kartel (ID: 2NUz5P42WqkxilbI8ocN76), Distance: 1.7320508075688772
Artist: Stylo G (ID: 7qPISKHhhKDLZTmYcX7bWd), Distance: 1.7320508075688772
Artist: Wayne Wonder (ID: 3kc5AFnL1TQQdNaMdSW2UO), Distance: 1.7320508075688772
Artist: Nesbeth (ID: 3Pi85h4xMLxui1v8ICQpw4), Distance: 1.7320508075688772
Artist: Konshens (ID: 3nwYsifpwrKmCIpw4i0HDW), Distance: 1.7320508075688772
Artist: UB40 (ID: 69MEO1AADKg1IZrq2XLzo5), Distance: 2.0
Artist: Dirty Heads (ID: 6GkJh85o22LfD2vgL9DP6f), Distance: 2.0
Artist: Damian Marley (ID: 3QJzdZJYIAcoET1GcfpNGi), Distance: 2.0


In [197]:
query_artist_3 = '74terC9ol9zMo8rfzhSOiG'

# Select the query artist row from the DataFrame
query_artist3 = artist_genre_df.loc[[query_artist_3]]

# Align both the query_artist and artist_genre_df to have the same columns
query3_artist_aligned = query_artist3.reindex(columns=artist_genre_df.columns, fill_value=0)

# Now perform the pairwise distance calculation
query3_euclidean_distances = pairwise_distances(query3_artist_aligned, artist_genre_df_aligned, metric='euclidean')[0]
query3_euclidean_top10_indices = query3_euclidean_distances.argsort()[:11]

print("Top 10 Artists Most Similar to Jakob Hellman, based on genre: \n")

for idx in query3_euclidean_top10_indices:
    artist_id = artist_genre_df.index[idx] 
    artist_name = id_to_name.get(artist_id, "Unknown Artist") 
    distance = query3_euclidean_distances[idx]  
    print(f"Artist: {artist_name} (ID: {artist_id}), Distance: {distance}")

Top 10 Artists Most Similar to Jakob Hellman, based on genre: 

Artist: Jakob Hellman (ID: 74terC9ol9zMo8rfzhSOiG), Distance: 0.0
Artist: Östen Med Resten (ID: 20nKo8C8M0JpIznby9Dv5p), Distance: 1.0
Artist: Helen Sjöholm (ID: 6M5pgjMS5dVG0hGRh5xADx), Distance: 1.0
Artist: Joakim Berg (ID: 5Z3qZD6rDNxloqapoa1QUC), Distance: 1.0
Artist: Pernilla Andersson (ID: 1muQ9itiu5KB1WiqehGoSy), Distance: 1.0
Artist: Melissa Horn (ID: 0tOdy7SqogILIjzGU2hSr5), Distance: 1.0
Artist: Niklas Strömstedt (ID: 3nEiRzdQNmkUgoknNt9IRu), Distance: 1.0
Artist: Lasse Berghagen (ID: 7kdGjv0IIQJcU2bth1yoqY), Distance: 1.0
Artist: Andrelli (ID: 5M2y5A6d5QZjw9JeKClagC), Distance: 1.4142135623730951
Artist: JUNG (ID: 1OHba9a2SK2KcJmJgkGQZO), Distance: 1.4142135623730951
Artist: Ana Diaz (ID: 4Lw6zDFtDp7WVblSAXcQdW), Distance: 1.4142135623730951


In [198]:
#dataframe displaying all results together

query_artist_ids = ['48WvrUGoijadXXCsGocwM4', '3dXC1YPbnQPsfHPVkm1ipj', '74terC9ol9zMo8rfzhSOiG']

table_data = []

for query_id in query_artist_ids:
    query_artist = artist_genre_df.loc[[query_id]]
    
    query_artist_aligned = query_artist.reindex(columns=artist_genre_df.columns, fill_value=0)
    
    distances = pairwise_distances(query_artist_aligned, artist_genre_df, metric='euclidean')[0]
    
    top10_indices = distances.argsort()[1:11]
    
    for idx in top10_indices:
        artist_id = artist_genre_df.index[idx]
        artist_name = id_to_name.get(artist_id, "Unknown Artist")
        distance = distances[idx]
        table_data.append([query_id, id_to_name[query_id], artist_id, artist_name, distance])

columns = ['Query Artist ID', 'Query Artist Name', 'Similar Artist ID', 'Similar Artist Name', 'Distance']
similar_artists_df = pd.DataFrame(table_data, columns=columns)

similar_artists_df.head(30)


,Query Artist ID,Query Artist Name,Similar Artist ID,Similar Artist Name,Distance
0,48WvrUGoijadXXCsGocwM4,Byklubben,49801AhfB2xOoOqtRZXv4H,Klikkmonopolet,1.000000
1,48WvrUGoijadXXCsGocwM4,Byklubben,18KsILOMx3thJRzBYBnKHW,J-Dawg,1.000000
2,48WvrUGoijadXXCsGocwM4,Byklubben,2zw4zwwb2LoNYssj3hdLoa,b.G,1.000000
3,48WvrUGoijadXXCsGocwM4,Byklubben,62InFTSRpk70tkIgzjLNTR,Snuskebazn,1.000000
4,48WvrUGoijadXXCsGocwM4,Byklubben,7IuE8jJQzeifyrcFYdrIWt,Heux,1.000000
5,48WvrUGoijadXXCsGocwM4,Byklubben,7krakxeyEmge6pzTthKNyg,Brødrene Bausa,1.000000
6,48WvrUGoijadXXCsGocwM4,Byklubben,5ksD5XT78Q3JpED82u9rNO,DJ Smaaland,1.000000
7,48WvrUGoijadXXCsGocwM4,Byklubben,1rQmmtMZFDzOadozdmep2k,Mad.S,1.000000
8,48WvrUGoijadXXCsGocwM4,Byklubben,5yM1po4NHvE2yE1Kf84LWJ,Hanzee,1.000000
9,48WvrUGoijadXXCsGocwM4,Byklubben,0l8OJzLFIV1OqbU4OhcZLu,Moberg,1.000000
